# 💿 Album Cover Vectorization

Goal: Process album cover images into high-dimensional vectors (embeddings) using SigLIP to enable semantic search in the Streamlit app.

In [ ]:
%load_ext autoreload
%autoreload 2

## Data

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
data_dir = Path("../../../data/covertartarchive/bm/")

In [ ]:
df_bm = pd.read_csv(data_dir / "df_final_with_covers.csv")
df_bm.head()

In [ ]:
df_bm[["band", "album", "cover_url"]].sample(5)

In [ ]:
df_bm["cover_file"] = df_bm["album_id"].apply(lambda x: f"{str(x)[-1]}/{x}.jpg")
df_bm = df_bm.dropna(subset=["cover_url"]).reset_index(drop=True)
img_paths = [data_dir / fname for fname in df_bm["cover_file"]]
df_bm.head()

In [ ]:
img_paths = [data_dir / fname for fname in df_bm["cover_file"]]
print(sum([p.exists() for p in img_paths]) / len(img_paths))

## Embedding covers

#### Model Initialization

We use SigLI via OpenCLIP. This model provides a shared embedding space for text and images, allowing us to find images based on natural language descriptions.

Exact implementation: https://huggingface.co/timm/ViT-B-16-SigLIP

In [ ]:
import torch
import open_clip

In [ ]:
# Device priority: CUDA > MPS > CPU
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

In [ ]:
# Load model
# model_name = "ViT-SO400M-14-SigLIP"
model_name = "ViT-B-16-SigLIP"
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name, pretrained="webli", device=device
)
# Load tokenizer
tokenizer = open_clip.get_tokenizer(model_name)

#### Data Loading & Preprocessing

Images are loaded and transformed to the specific resolution (224x224) required by the SigLIP Vision Transformer.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.auto import tqdm
import numpy as np

In [ ]:
class AlbumDataset(Dataset):
    """
    A standard PyTorch Dataset for album covers.

    Assumes all image files in 'paths' exist and are valid.
    Loads images, converts them to RGB, and applies SigLIP preprocessing.
    """

    def __init__(self, paths, preprocess):
        """
        Args:
            paths (list): List of pathlib.Path objects.
            preprocess (callable): The image transformation pipeline.
        """
        self.paths = paths
        self.preprocess = preprocess

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        """
        Returns:
            tuple: (preprocessed_tensor, index)
        """
        path = self.paths[idx]
        image = Image.open(path).convert("RGB")
        return self.preprocess(image), idx

In [ ]:
dataset = AlbumDataset(img_paths, preprocess)
loader = DataLoader(
    dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False
)

### Embedding Generation

We iterate through the dataset in batches to generate 768-dimensional vectors using the SigLIP encoder.

Normalization: All vectors are L2 normalized (default p=2). This ensures their magnitude is 1.0, and that a simple dot product gives the Cosine Similarity.

In [ ]:
valid_indices = []
embeddings = []

model.eval()
with torch.no_grad():
    for images, indices in tqdm(loader, desc=f"Encoding on {device}"):
        images = images.to(device)

        # Forward pass
        features = model.encode_image(images)

        # Normalize
        features /= features.norm(dim=-1, keepdim=True)

        embeddings.append(features.cpu().numpy())
        valid_indices.extend(indices.tolist())

In [ ]:
# Final matrix
all_embs = np.vstack(embeddings)

In [ ]:
# 1. Initialize the column
df_bm["embedding"] = None

# 2. Convert the matrix into a list of individual arrays
# This allows Pandas to place one array into each cell
embedding_list = list(all_embs)

# 3. Map them back to the correct rows
df_bm.loc[valid_indices, "embedding"] = pd.Series(list(all_embs), index=valid_indices)

# 4. Create a helper boolean for easy filtering later
df_bm["has_embedding"] = df_bm["embedding"].notna()


# 5. Check how many were mapped
total_rows = len(df_bm)
mapped_count = df_bm["has_embedding"].sum()
# Calculate percentage
pct_success = (mapped_count / total_rows) * 100

print(
    f"Successfully mapped {mapped_count} embeddings to the DataFrame "
    f"({pct_success:.2f}% success rate)."
)

In [ ]:
all_embs.shape

In [ ]:
# Save to Pickle: Preserves NumPy arrays natively and requires no extra dependencies.
# File size: ~16MB
cols_to_save = ["band", "album", "year", "cover_url", "embedding", "album_id"]
df_bm[df_bm["has_embedding"]][cols_to_save].to_pickle(
    "../data/bm_covers_with_embeddings.pkl"
)

### Semantic Search Validation

Verify SigLIP's ability to retrieve relevant album covers using descriptive text queries.

In [ ]:
df = pd.read_pickle("../data/bm_covers_with_embeddings.pkl")
df.head()

In [ ]:
def encode_text_query(text, model, tokenizer, device):
    """Encode text query using SigLIP-specific tokenizer"""
    text_tokens = tokenizer([text]).to(device)

    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        return text_features.cpu().numpy()

In [ ]:
def find_top_albums(query_text, df, model, tokenizer, device, top_k=10):
    """Search the dataframe and return top matches"""
    # 1. Get query vector
    query_emb = encode_text_query(query_text, model, tokenizer, device)

    # 2. Extract matrix of valid embeddings [N, 768]
    matrix = np.vstack(df["embedding"].values)

    # 3. Calculate scores (Dot product = Cosine Similarity for normalized vectors)
    scores = (query_emb @ matrix.T).flatten()

    # 4. Get top indices
    best_indices = scores.argsort()[::-1][:top_k]

    # 5. Return the top rows with their scores
    results = df.iloc[best_indices].copy()
    results["search_score"] = scores[best_indices]
    return results

In [ ]:
# --- Semantic Search Sandbox ---
query = "a dark castle"
top_10 = find_top_albums(query, df, model, tokenizer, device)

In [ ]:
from IPython.display import display, HTML


def display_top_3_html(results_df):
    html_str = '<div style="display: flex; gap: 20px; align-items: flex-start;">'

    for _, row in results_df.head(3).iterrows():
        html_str += f"""
        <div style="flex: 1; text-align: center; max-width: 250px;">
            <img src="{row["cover_url"]}" style="width: 100%; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
            <p style="margin: 8px 0 0 0; font-weight: bold; font-family: sans-serif;">{row["band"]}</p>
            <p style="margin: 2px 0; font-style: italic; font-family: sans-serif;">{row["album"]}</p>
            <p style="color: #666; font-size: 0.8em; font-family: sans-serif;">Score: {row["search_score"]:.3f}</p>
        </div>
        """

    html_str += "</div>"
    display(HTML(html_str))


display_top_3_html(top_10)